# Future Home Location Project

This notebook only includes the cleaning and merging of the data used in the app. Look in the Future_Hometown_Finder_Application folder for the application code. 

The actual application can be found here: https://future-hometown-finder.herokuapp.com/. If you do choose to use the application, please only run it a few times. Google's free custom search api limits the number of api calls per day so if you make too many request the app will stop returning images.

* https://simplemaps.com/data/us-cities
* https://www.zillow.com/research/data/
* https://data.world/mattwinter225/2015-usa-weather-avg-max-min
* https://data.world/ucr/crime-in-us-2016-offenses-known-by-state-by-city
* https://www.shepscenter.unc.edu/programs-projects/rural-health/data/

# Imports

In [1]:
import numpy as np
import pandas as pd
import os
import glob

from sklearn.impute import KNNImputer

# Loading and Cleaning Data

## Cities Data

In [2]:
cities = pd.read_csv("~/data/Home_Location_Project/uscities.zip", 
                     usecols=["city", "state_id", "state_name", "county_name", "population", 
                              "density", "ranking", "zips", "military", "lat", "lng"])

* **city:** The name of the city/town.									
* **state_id:** The state or territory's USPS postal abbreviation.			
* **state_name:** The name of the state or territory that contains the city/town.				
* **county_name:** The name of the primary county (or equivalent) that contains the city/town.			
* **population:** An estimate of the city's urban population. (2019).				
* **density:** The estimated population per square kilometer.							
* **ranking:** An integer from 1-5 that captures the importance of a city (1 is most important, 5 least important).			
* **zips:** A string containing all five-digit zip codes in the city/town, delimited by a space. Learn more.				
* **military:** TRUE if this place is a military establishment such as a fort or base.				
* **lat:**	The latitude of the city/town.			
* **lng:**	The longitude of the city/town.

In [3]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,military,ranking,zips
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,False,1,11229 11226 11225 11224 11222 11221 11220 1138...
1,Los Angeles,CA,California,Los Angeles,34.1139,-118.4068,12750807,3276,False,1,90291 90293 90292 91316 91311 90037 90031 9000...
2,Chicago,IL,Illinois,Cook,41.8373,-87.6862,8604203,4574,False,1,60018 60649 60641 60640 60643 60642 60645 6064...
3,Miami,FL,Florida,Miami-Dade,25.7839,-80.2102,6445545,5019,False,1,33129 33125 33126 33127 33128 33149 33144 3314...
4,Dallas,TX,Texas,Dallas,32.7936,-96.7662,5743938,1526,False,1,75287 75098 75233 75254 75251 75252 75253 7503...
...,...,...,...,...,...,...,...,...,...,...,...
28333,Gross,NE,Nebraska,Boyd,42.9461,-98.5697,2,6,False,3,68719
28334,Lotsee,OK,Oklahoma,Tulsa,36.1334,-96.2091,2,39,False,3,74063
28335,The Ranch,MN,Minnesota,Mahnomen,47.3198,-95.6952,2,2,False,3,56557
28336,Shamrock,OK,Oklahoma,Creek,35.9113,-96.5772,2,2,False,3,74068


In [4]:
cities.describe()

,lat,lng,population,density,ranking
count,28338.000000,28338.000000,2.833800e+04,28338.000000,28338.000000
mean,38.613995,-92.972393,1.426071e+04,515.410615,2.945938
std,5.870832,15.596099,1.898112e+05,786.011155,0.233816
min,17.955900,-176.629500,1.000000e+00,0.000000,1.000000
25%,35.126875,-98.193975,3.300000e+02,117.000000,3.000000
50%,39.314800,-90.250100,1.089000e+03,297.000000,3.000000
75%,41.743725,-81.933650,4.485500e+03,603.000000,3.000000
max,71.272800,173.123000,1.871322e+07,34277.000000,3.000000


In [5]:
# Remove military bases.
cities = cities[~cities["military"]]

# Remove places with low population. Many of these places have no housing avaliable.
cities = cities[cities["population"]>=20]

# Get a single zip code if city has multiple
cities["single_zip"] = [pd.Series(cities["zips"][row].split(" ")).astype(int)[0] for row in cities.index]

cities.drop(["military", "zips"], axis=1, inplace=True)

# No missing values
cities.isna().sum()

city           0
state_id       0
state_name     0
county_name    0
lat            0
lng            0
population     0
density        0
ranking        0
single_zip     0
dtype: int64

In [6]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,ranking,single_zip
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,1,11229
1,Los Angeles,CA,California,Los Angeles,34.1139,-118.4068,12750807,3276,1,90291
2,Chicago,IL,Illinois,Cook,41.8373,-87.6862,8604203,4574,1,60018
3,Miami,FL,Florida,Miami-Dade,25.7839,-80.2102,6445545,5019,1,33129
4,Dallas,TX,Texas,Dallas,32.7936,-96.7662,5743938,1526,1,75287
...,...,...,...,...,...,...,...,...,...,...
28100,Ferry,AK,Alaska,Denali,64.0496,-148.9205,20,0,3,99760
28101,Red Devil,AK,Alaska,Bethel,61.7735,-157.3460,20,0,3,99656
28102,Salmon Creek,CA,California,Sonoma,38.3463,-123.0595,20,6,3,94923
28103,Lindy,NE,Nebraska,Knox,42.7351,-97.7499,20,7,3,68718


## Housing Data

In [7]:
housing = pd.read_csv("~/data/Home_Location_Project/City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.zip", 
                      usecols=["RegionName", "RegionType", 
                                "State", "CountyName", "2021-06-30"])

housing.rename(columns={"2021-06-30":"ZHVI_2021-06-30"}, inplace=True)

* __SizeRank:__ Rank the size of the city. Lower rank means larger city.
* __RegionName:__ The name of the region (city)
* __RegionType:__ The type of region referenced in the RegionName column.
* __State:__ The name of the state.
* __CountyName:__ The name of the county.
* __ZHVI_2021-06-30:__ Zillow Home Value Index (ZHVI). A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range.


In [8]:
housing

,RegionName,RegionType,State,CountyName,ZHVI_2021-06-30
0,New York,City,NY,Queens County,662535.0
1,Los Angeles,City,CA,Los Angeles County,882150.0
2,Houston,City,TX,Harris County,222540.0
3,Chicago,City,IL,Cook County,296603.0
4,San Antonio,City,TX,Bexar County,221886.0
...,...,...,...,...,...
26121,Winton,City,MN,Saint Louis County,81369.0
26122,Eastabuchie,City,MS,Jones County,84927.0
26123,Dean,City,TX,Clay County,204499.0
26124,Pulaski,City,GA,Candler County,112883.0


In [9]:
housing.rename(columns={"RegionName":"City", "ZHVI_2021-06-30":"Median_Home_Price"}, inplace=True)

housing["County"] = housing["CountyName"].str.extract(r"(.*) County")

housing.describe()

,Median_Home_Price
count,2.612600e+04
mean,2.416043e+05
std,3.024785e+05
min,1.284900e+04
25%,1.080165e+05
50%,1.728240e+05
75%,2.852778e+05
max,2.356101e+07


In [10]:
housing_price_by_county = housing.groupby(["County", "State"]).median()
housing_price_by_county

Median_Home_Price
County    State                   
Abbeville SC              118965.0
Accomack  VA              164836.5
Ada       ID              512962.0
Adair     IA              104032.0
          KY              105202.0
...                            ...
Yuba      CA              335136.5
Yuma      AZ              188771.0
          CO              195317.0
Zapata    TX               81628.0
Zavala    TX               68803.0

[2841 rows x 1 columns]

In [11]:
housing_price_by_city = housing.groupby(["City", "State"]).median()
housing_price_by_city

Median_Home_Price
City       State                   
Aaronsburg PA              199933.0
Abbeville  AL               96798.0
           GA               80680.0
           LA              148282.0
           MS              136239.0
...                             ...
Zumbrota   MN              268685.0
Zuni       VA              257827.0
Zurich     KS               74350.0
Zwingle    IA              228174.0
Zwolle     LA              119790.0

[26061 rows x 1 columns]

In [12]:
cities = cities.merge(
    right=housing_price_by_county, 
    left_on=["county_name", "state_id"], 
    right_on=["County", "State"],
    how="left"
)

cities = cities.merge(
    right=housing_price_by_city, 
    left_on=["city", "state_id"], 
    right_on=["City", "State"],
    how="left"
)

In [13]:
cities["Median_Home_Price_x"][cities["Median_Home_Price_x"].isna()] = 0
cities["Median_Home_Price_y"][cities["Median_Home_Price_y"].isna()] = 0

# if we have data for the home prices in the city, then we use that, else we use the home prices of the city's county
cities["estimated_home_price"] = \
    cities["Median_Home_Price_x"] * (cities["Median_Home_Price_y"]==0) + \
    cities["Median_Home_Price_y"] * ~(cities["Median_Home_Price_y"]==0)
    
cities.drop(["Median_Home_Price_x", "Median_Home_Price_y"], axis=1, inplace=True)
cities = cities[cities["estimated_home_price"] != 0]

# SettingWithCopyWarning can be ignored. These operations do modify the cities dataframe as intended

C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,ranking,single_zip,estimated_home_price
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,1,11229,662535.0
1,Los Angeles,CA,California,Los Angeles,34.1139,-118.4068,12750807,3276,1,90291,882150.0
2,Chicago,IL,Illinois,Cook,41.8373,-87.6862,8604203,4574,1,60018,296603.0
3,Miami,FL,Florida,Miami-Dade,25.7839,-80.2102,6445545,5019,1,33129,407242.0
4,Dallas,TX,Texas,Dallas,32.7936,-96.7662,5743938,1526,1,75287,266742.0
...,...,...,...,...,...,...,...,...,...,...,...
28012,Sholes,NE,Nebraska,Wayne,42.3348,-97.2946,20,56,3,68771,153134.5
28013,Spring Garden,CA,California,Plumas,39.9012,-120.7935,20,11,3,95971,271865.0
28017,Salmon Creek,CA,California,Sonoma,38.3463,-123.0595,20,6,3,94923,822862.5
28018,Lindy,NE,Nebraska,Knox,42.7351,-97.7499,20,7,3,68718,73573.0


## Weather Data

* __STATION_NAME:__ The name of the weather station
* __AvgTemp:__ The average temperature recorded by the weather station on that day
* __MaxTemp:__ The maximum temperature recorded by the weather station on that day
* __MinTemp:__ The minimum temperature recorded by the weather station on that day
* __StateName:__ The name of the state
* __Zip:__ Zip code where the station is located

In [15]:
temperature = pd.read_csv("~/data/Home_Location_Project/2015_USA_Weather_Data_FINAL.zip", sep=';', 
                      usecols=["STATION_NAME", "AvgTemp", "MaxTemp", 
                                "MinTemp", "StateName", "Zip"])

In [16]:
temperature

,STATION_NAME,AvgTemp,MaxTemp,MinTemp,StateName,Zip
0,WORCESTER MA US,55.0,58.0,47.0,Massachusetts,1602.0
1,WORCESTER MA US,47.0,49.0,44.0,Massachusetts,1602.0
2,WORCESTER MA US,45.0,49.0,42.0,Massachusetts,1602.0
3,WORCESTER MA US,47.0,53.0,41.0,Massachusetts,1602.0
4,WORCESTER MA US,49.0,59.0,44.0,Massachusetts,1602.0
...,...,...,...,...,...,...
821124,TALLGRASS PRAIRIE KANSAS KS US,78.0,78.0,78.0,Kansas,66869.0
821125,TALLGRASS PRAIRIE KANSAS KS US,94.0,94.0,94.0,Kansas,66869.0
821126,TALLGRASS PRAIRIE KANSAS KS US,83.0,99.0,66.0,Kansas,66869.0
821127,TALLGRASS PRAIRIE KANSAS KS US,71.0,88.0,61.0,Kansas,66869.0


In [17]:
# Convert zips to integers
temperature["Zip"] = temperature["Zip"].astype(int)

# Note that we do not need to clean bad zipcodes, the join with cities will do that automatically

In [18]:
# Create a df w/ median temperature by state
temperature_by_state = temperature.groupby("StateName").median().drop("Zip", axis=1) # Carolina state +> Carolina, Puerto Rico
temperature_by_state

,AvgTemp,MaxTemp,MinTemp
StateName,,,
0,79.0,85.0,74.0
Alabama,68.0,79.0,59.0
Alaska,36.0,43.0,30.0
Arizona,55.0,68.0,43.0
Arkansas,62.0,73.0,52.0
California,57.0,70.0,47.0
Carolina,79.0,86.0,74.0
Colorado,40.0,52.0,31.0
Connecticut,55.0,64.0,45.0


In [19]:
temperature_by_zip = temperature.groupby("Zip").median()
temperature_by_zip

,AvgTemp,MaxTemp,MinTemp
Zip,,,
0,46.0,56.0,37.0
979,79.0,86.0,74.0
1602,51.0,59.0,42.0
2536,53.0,60.0,46.5
2886,54.0,63.0,44.0
...,...,...,...
99840,36.0,40.0,33.0
99841,42.0,49.0,38.0
99901,51.0,58.0,44.0


In [20]:
cities = cities.merge(
    right=temperature_by_state, 
    left_on="state_name", 
    right_on="StateName", 
    how="inner"
)

cities = cities.merge(
    right=temperature_by_zip,
    left_on="single_zip", 
    right_on="Zip", 
    how="left"
)

In [21]:
# For estimated nighttime temperature:
cities["MinTemp_x"][cities["MinTemp_x"].isna()] = 0
cities["MinTemp_y"][cities["MinTemp_y"].isna()] = 0
# if we have data for the temp in the city's zipcode, then we use that, else we use the state's temp
cities["estimated_nighttime_temp"] = cities["MinTemp_x"] * (cities["MinTemp_y"]==0) + \
                                    cities["MinTemp_y"] * ~(cities["MinTemp_y"]==0)


# For estimated daytime temperature
cities["MaxTemp_x"][cities["MaxTemp_x"].isna()] = 0
cities["MaxTemp_y"][cities["MaxTemp_y"].isna()] = 0
# if we have data for the temp in the city's zipcode, then we use that, else we use the state's temp
cities["estimated_daytime_temp"] = cities["MaxTemp_x"] * (cities["MaxTemp_y"]==0) + \
                                    cities["MaxTemp_y"] * ~(cities["MaxTemp_y"]==0)


cities.drop(["AvgTemp_x", "MaxTemp_x", "MinTemp_x", "AvgTemp_y", "MaxTemp_y", "MinTemp_y"], axis=1, inplace=True)

# SettingWithCopyWarning can be ignored. These operations do modify the cities dataframe as intended

C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added

In [22]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,ranking,single_zip,estimated_home_price,estimated_nighttime_temp,estimated_daytime_temp
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,1,11229,662535.0,41.0,62.0
1,Queens,NY,New York,Queens,40.7498,-73.7976,2230722,7885,1,11361,662535.0,41.0,62.0
2,Buffalo,NY,New York,Erie,42.9016,-78.8487,914341,2441,2,14208,186970.0,41.0,62.0
3,Rochester,NY,New York,Monroe,43.1680,-77.6162,703952,2220,2,14608,132486.0,41.0,62.0
4,Albany,NY,New York,Albany,42.6664,-73.7987,586383,1740,2,12208,240961.0,41.0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26658,Little America,WY,Wyoming,Sweetwater,41.5470,-109.8642,45,6,3,82929,160925.0,30.0,50.0
26659,Boulder,WY,Wyoming,Sublette,42.7461,-109.7067,32,6,3,82923,329449.0,28.5,47.0
26660,Jeffrey City,WY,Wyoming,Fremont,42.4808,-107.8256,29,0,3,82520,103570.0,30.0,51.0
26661,Ryan Park,WY,Wyoming,Carbon,41.3131,-106.4895,26,4,3,82331,192719.0,32.0,52.0


## Crime

In [23]:
datapath = "C:\\Users\\mitch\\data\\Home_Location_Project\\ucr-crime-in-us-2016-offenses-known-by-state-by-city\\"
allfiles = glob.glob(datapath + "*.xls")
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", 
          "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", 
          "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
          "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York",
          "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
          "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia",
          "Washington", "West Virginia", "Wisconsin", "Wyoming"]
i = 0
crime_dict = {} # States are keys and dataframes are values

for f in allfiles:
    crime_dict[states[i]] = pd.read_excel(f, header=4, skipfooter=6)
    
    # add a state column
    crime_dict[states[i]]["state"] = states[i]
    
    i+=1

In [24]:
crime = pd.concat(crime_dict, ignore_index=True).fillna(0)

* __City:__ Name of city
* __Population:__ The population of the city


* __Violent\ncrime:__ Number of violent crimes (sum of the following 5 categories)
* __Murder and\nnonnegligent\nmanslaughter:__ Number of murders and nonnegligent manslaughters
* __Rape\n(revised\ndefinition)1:__ Number of rapes (New definition)
* __Rape\n(legacy\ndefinition)2:__ Number of rapes (Old definition)
* __Robbery:__ Number of robberies
* __Aggravated\nassault:__ Number of aggravated assault


* __Property\ncrime:__ Number of property crimes (sum of the following 3 categories)
* __Burglary:__ Number of burglaries
* __Larceny-\ntheft:__ Number of Larcenies
* __Motor\nvehicle\ntheft:__ Number of motor vehicle thefts


* __Arson3:__ Number of arsons (Some files uses Arson3, others use Arson)
* __Arson:__ Number of arsons (Some files uses Arson3, others use Arson)


* __state:__ The state in which the city is located

In [25]:
crime

,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3,state,Arson
0,Abbeville,2608.0,11.0,0,1.0,0.0,0.0,10.0,51.0,12.0,34.0,5.0,0.0,Alabama,0.0
1,Adamsville,4377.0,19.0,0,0.0,0.0,10.0,9.0,250.0,33.0,201.0,16.0,0.0,Alabama,0.0
2,Addison,738.0,1.0,0,0.0,0.0,0.0,1.0,14.0,1.0,11.0,2.0,0.0,Alabama,0.0
3,Alabaster,33040.0,97.0,1,2.0,0.0,2.0,92.0,488.0,58.0,411.0,19.0,0.0,Alabama,0.0
4,Albertville,21525.0,29.0,0,5.0,0.0,10.0,14.0,721.0,190.0,462.0,69.0,0.0,Alabama,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9420,Riverton,10905.0,47.0,1,8.0,0.0,4.0,34.0,610.0,52.0,521.0,37.0,0.0,Wyoming,1.0
9421,Rock Springs,24161.0,79.0,2,35.0,0.0,2.0,40.0,491.0,58.0,406.0,27.0,0.0,Wyoming,0.0
9422,Saratoga,1675.0,0.0,0,0.0,0.0,0.0,0.0,19.0,6.0,11.0,2.0,0.0,Wyoming,0.0
9423,Sheridan,17956.0,17.0,2,0.0,0.0,1.0,14.0,362.0,54.0,295.0,13.0,0.0,Wyoming,2.0


In [26]:
# Drop columns we dont need
crime.drop(["Murder and\nnonnegligent\nmanslaughter", "Rape\n(revised\ndefinition)1", "Rape\n(legacy\ndefinition)2",
           "Robbery", "Aggravated\nassault", "Burglary", "Larceny-\ntheft", "Motor\nvehicle\ntheft"], axis=1, inplace=True)

In [27]:
total_crimes = crime[["Violent\ncrime", "Property\ncrime", "Arson3", "Arson"]].sum(axis=1)

crimes_per_thousand_people = (1000 * total_crimes) / (crime["Population"]+1) # some have pop==0 for some reason

# If crime rate is less than 5/1000 people, assume data is bad and set it to 15/1000
crimes_per_thousand_people[crimes_per_thousand_people < 5] = 15
crimes_per_thousand_people

# Cap crime rate at 100 per 1000 people
crimes_per_thousand_people[crimes_per_thousand_people > 100] = 100
crimes_per_thousand_people

0       23.763894
1       61.443582
2       20.297700
3       17.705275
4       34.841587
          ...    
9420    60.333761
9421    23.590762
9422    11.336516
9423    21.217353
9424    11.627907
Length: 9425, dtype: float64

In [28]:
crime["crimes_per_thousand_people"] = crimes_per_thousand_people
crime

,City,Population,Violent\ncrime,Property\ncrime,Arson3,state,Arson,crimes_per_thousand_people
0,Abbeville,2608.0,11.0,51.0,0.0,Alabama,0.0,23.763894
1,Adamsville,4377.0,19.0,250.0,0.0,Alabama,0.0,61.443582
2,Addison,738.0,1.0,14.0,0.0,Alabama,0.0,20.297700
3,Alabaster,33040.0,97.0,488.0,0.0,Alabama,0.0,17.705275
4,Albertville,21525.0,29.0,721.0,0.0,Alabama,0.0,34.841587
...,...,...,...,...,...,...,...,...
9420,Riverton,10905.0,47.0,610.0,0.0,Wyoming,1.0,60.333761
9421,Rock Springs,24161.0,79.0,491.0,0.0,Wyoming,0.0,23.590762
9422,Saratoga,1675.0,0.0,19.0,0.0,Wyoming,0.0,11.336516
9423,Sheridan,17956.0,17.0,362.0,0.0,Wyoming,2.0,21.217353


In [29]:
# Drop the rest of the columns so we can merge just what we need
crime_by_city = crime.drop(["Population", "Violent\ncrime", "Property\ncrime", "Arson3", "Arson"], axis=1)
crime_by_city

,City,state,crimes_per_thousand_people
0,Abbeville,Alabama,23.763894
1,Adamsville,Alabama,61.443582
2,Addison,Alabama,20.297700
3,Alabaster,Alabama,17.705275
4,Albertville,Alabama,34.841587
...,...,...,...
9420,Riverton,Wyoming,60.333761
9421,Rock Springs,Wyoming,23.590762
9422,Saratoga,Wyoming,11.336516
9423,Sheridan,Wyoming,21.217353


In [30]:
cities = cities.merge(
    right=crime_by_city,
    left_on=["city", "state_name"], 
    right_on=["City", "state"],
    how="left"
)

cities.drop(["City", "state"], axis=1, inplace=True)

In [31]:
# Estimate missing crime rates based on the latitude and longitude 
imputer = KNNImputer(n_neighbors = 3, weights="distance", copy = False)
imputer_helper_data = cities[["lat", "lng", "crimes_per_thousand_people"]]
imputer.fit_transform(imputer_helper_data)
cities["crimes_per_thousand_people"] = imputer_helper_data["crimes_per_thousand_people"]

In [32]:
cities["crimes_per_thousand_people"] = round(cities["crimes_per_thousand_people"], 2)

In [33]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,ranking,single_zip,estimated_home_price,estimated_nighttime_temp,estimated_daytime_temp,crimes_per_thousand_people
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,1,11229,662535.0,41.0,62.0,20.36
1,Queens,NY,New York,Queens,40.7498,-73.7976,2230722,7885,1,11361,662535.0,41.0,62.0,16.45
2,Buffalo,NY,New York,Erie,42.9016,-78.8487,914341,2441,2,14208,186970.0,41.0,62.0,53.15
3,Rochester,NY,New York,Monroe,43.1680,-77.6162,703952,2220,2,14608,132486.0,41.0,62.0,46.55
4,Albany,NY,New York,Albany,42.6664,-73.7987,586383,1740,2,12208,240961.0,41.0,62.0,43.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26658,Little America,WY,Wyoming,Sweetwater,41.5470,-109.8642,45,6,3,82929,160925.0,30.0,50.0,18.94
26659,Boulder,WY,Wyoming,Sublette,42.7461,-109.7067,32,6,3,82923,329449.0,28.5,47.0,23.30
26660,Jeffrey City,WY,Wyoming,Fremont,42.4808,-107.8256,29,0,3,82520,103570.0,30.0,51.0,42.71
26661,Ryan Park,WY,Wyoming,Carbon,41.3131,-106.4895,26,4,3,82331,192719.0,32.0,52.0,16.79


## Hospitals

In [34]:
# Hospitals dataframe contains the location of hospitals in the United States. 
# Other information such as hospital name is not needed, thus it is not read in.
hospitals = pd.read_excel("~/data/Home_Location_Project/HospitalList2020daFINAL.xlsx", 
                         usecols=["CITY", "STATE", "ZIP"])

C:\Users\mitch\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [35]:
hospitals["has_hospital"] = 1

In [36]:
hospitals.drop_duplicates()

,CITY,STATE,ZIP,has_hospital
0,DOTHAN,AL,36301,1
1,BOAZ,AL,35957,1
2,FLORENCE,AL,35630,1
3,OPP,AL,36467,1
4,LUVERNE,AL,36049,1
...,...,...,...,...
4606,THE WOODLANDS,TX,77385,1
4607,HORIZON CITY,TX,79928,1
4608,SAN ANTONIO,TX,78223,1
4609,PFLUGERVILLE,TX,78660,1


In [37]:
cities = cities.merge(
    right=hospitals.drop(["CITY", "STATE"], axis=1), 
    left_on="single_zip", 
    right_on="ZIP", 
    how="left"
)
cities.drop("ZIP", axis=1, inplace=True)

cities = pd.merge(
    left=cities,
    right=hospitals.drop("ZIP", axis=1),
    left_on=[cities.city.str.lower(), cities.state_id], 
    right_on=[hospitals.CITY.str.lower(), hospitals.STATE],
    how="left"
)
cities.drop(["CITY", "STATE"], axis=1, inplace=True)

In [38]:
# if there is a hospital in the city or zipcode, then has_hospital = True
cities["has_hospital"] = (~cities["has_hospital_x"].isna() | ~cities["has_hospital_y"].isna()) > 0
cities = cities.drop_duplicates().reset_index()
cities.drop(["index", "key_0", "key_1", "has_hospital_x", "has_hospital_y"], axis=1, inplace=True)

In [39]:
cities

,city,state_id,state_name,county_name,lat,lng,population,density,ranking,single_zip,estimated_home_price,estimated_nighttime_temp,estimated_daytime_temp,crimes_per_thousand_people,has_hospital
0,New York,NY,New York,New York,40.6943,-73.9249,18713220,10715,1,11229,662535.0,41.0,62.0,20.36,True
1,Queens,NY,New York,Queens,40.7498,-73.7976,2230722,7885,1,11361,662535.0,41.0,62.0,16.45,False
2,Buffalo,NY,New York,Erie,42.9016,-78.8487,914341,2441,2,14208,186970.0,41.0,62.0,53.15,True
3,Rochester,NY,New York,Monroe,43.1680,-77.6162,703952,2220,2,14608,132486.0,41.0,62.0,46.55,True
4,Albany,NY,New York,Albany,42.6664,-73.7987,586383,1740,2,12208,240961.0,41.0,62.0,43.74,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26658,Little America,WY,Wyoming,Sweetwater,41.5470,-109.8642,45,6,3,82929,160925.0,30.0,50.0,18.94,False
26659,Boulder,WY,Wyoming,Sublette,42.7461,-109.7067,32,6,3,82923,329449.0,28.5,47.0,23.30,False
26660,Jeffrey City,WY,Wyoming,Fremont,42.4808,-107.8256,29,0,3,82520,103570.0,30.0,51.0,42.71,False
26661,Ryan Park,WY,Wyoming,Carbon,41.3131,-106.4895,26,4,3,82331,192719.0,32.0,52.0,16.79,False


In [40]:
# Save this dataframe as a csv
cities.drop(["state_id", "county_name", "lat", "lng", "ranking"], inplace=True, axis=1)

compression_opts = dict(method='zip', archive_name='Future_Home_Location_Data.csv')
cities.to_csv('Future_Home_Location_Data.zip', index=False, compression=compression_opts) 

# Recommended  Cities Function

This function is used to calculate which cities best match user specified information. This is the function used in the actual application.

In [41]:
def recommend_cities(data, state, ideal_pop_size, ideal_pop_density, ideal_home_price, ideal_daytime_temperature, hospital):
    """
    data: The dataset.
    state: The user's preferred state.
    ideal_pop_size: The user's ideal city/town population size.
    ideal_pop_density: Ranges in km**2: Low(0,250), Average(250, 750), High(750, oo), No Preference
    ideal_home_price: The user's ideal home price.
    ideal_daytime_temperature: The user's ideal daytime temperature.
    hospital: Whether or not we care if a hospital is nearby (within the same city/zipcode).
    """
    if (state != "No Preference"):
        data = data[data["state_name"] == state]
        
    if (ideal_pop_density == "Low"):
        data = data[data["density"] <= 250]
    elif (ideal_pop_density == "Average"):
        data = data[250 <= data["density"]]
        data = data[data["density"] <= 750]
    elif (ideal_pop_density == "High"):
        data = data[data["density"] >= 750]

    data["score"] = (abs(data["population"]-ideal_pop_size) / (data["population"]+ideal_pop_size))**2 +\
                    (abs(data["estimated_home_price"]-ideal_home_price) / (data["estimated_home_price"]+ideal_home_price)) +\
                    (abs(data["estimated_daytime_temp"]-ideal_daytime_temperature) / (data["estimated_daytime_temp"]+ideal_daytime_temperature)) +\
                    (data["crimes_per_thousand_people"] / 100)

    # Multiply the current score by 1.5 if there isn't a local hospital
    if(hospital):
        data["score"] = data["score"] + 0.5 * \
            data["score"] * ~data["has_hospital"]

    return data.sort_values("score").reset_index().drop(["index", "score"], axis=1).head(20)

In [42]:
recommend_cities(
    data = cities, 
    ideal_pop_size = 50000, 
    ideal_pop_density = "High",
    ideal_home_price = 350000, 
    ideal_daytime_temperature = 68, 
    hospital = True,
    state="No Preference"
)

C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,city,state_name,population,density,single_zip,estimated_home_price,estimated_nighttime_temp,estimated_daytime_temp,crimes_per_thousand_people,has_hospital
0,Buffalo Grove,Illinois,40494,1648,60089,339497.0,47.0,68.0,5.32,False
1,Hoffman Estates,Illinois,50932,933,60010,312821.0,47.0,68.0,9.28,True
2,Wheaton,Illinois,52745,1801,60187,379256.0,47.0,68.0,6.78,False
3,Mount Prospect,Illinois,53719,1934,60056,351236.0,47.0,68.0,10.87,False
4,Bartlett,Illinois,40647,996,60103,325252.0,47.0,68.0,6.85,False
5,Downers Grove,Illinois,49057,1304,60515,377341.0,47.0,68.0,13.77,True
6,Arlington Heights,Illinois,74760,1738,60005,388121.0,47.0,68.0,10.29,True
7,Perth Amboy,New Jersey,51390,4255,8861,349585.0,50.0,68.0,19.33,True
8,State College,Pennsylvania,87723,3572,16802,329175.0,45.0,66.0,9.17,True
9,Fishers,Indiana,95310,1037,46038,344323.0,46.0,67.0,10.61,True
